# Story Teller

In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import requests
import os
import asyncio

In [3]:
load_dotenv(override=True)

True

## Agents

In [4]:
instructions_place = """
You are a story-telling agent responsible for picking the place where the story is set. This is your only task, you just provide
a location and nothing more. The location can be a real geographical place, like for example 'Japan', 'New York' or 'the Alps', but it can 
be also a fictitious place like for example 'the Lake of Doom', 'the Deserted Mountains' or something like that. Another option is
to deliver just a general name like for example 'the big city behind the forest' or 'a little cozy village at the foot of the big mountain'.
You can create a descriptive name like this or just return a simple one-word name. Do not add any explanation or comments, just deliver the name
of the place as a word or short expression.
"""

instructions_time = """
You are a story-telling agent responsible for picking the time where the story is set. This is your only task, you just provide
a time and nothing more. The time can be a date, like for example 'October 14, 2014', '1945-08-20' or 'May of 2020', but it can 
be also an expression containing a real historical event, like for example 'during the second world war', 'before the island was invaded by the Vikings', 
or something like that. Another option is to deliver just a general time expression like for example 'at the beginning of summer' or 
'when dinosaurs ruled the world'. You can create a descriptive time expression like this or just return a simple date. Do not add any explanation 
or comments, just deliver the time as a date or short time expression.
"""

instructions_characters = """
You are a story-telling agent responsible for picking the characters featured in the story. This is your only task, you just provide
a list of characters and nothing more. The list should contain between one and three characters, no more. The characters can be names,
like for example 'George', 'Anne Wilson' or 'Mr. Smith', but they can be also general terms like 'a girl', 'a young man' or something like that. 
If there are more than 1 character, you can use terms showing their relationships with the first character. Such a list could look like so:
['little John', 'his sister', 'their mother']. You can also use collective characters like 'his friends', 'the army' or 'French people'.
You can create a descriptive name like this or just return a simple one-word name for each character. Do not add any explanation or comments, 
just deliver the list of 1-3 characters.
"""

instructions_items = """
You are a story-telling agent responsible for picking the items used in the story. This is your only task, you just provide
a list of items and nothing more. The list should contain between one and three items, no more. The characters can be names of animals, plants,
or any other objects, like for example 'an elephant', 'a palm tree' or 'a car', but they can be also more descriptive names like 'an old red bicycle', 
'a beautiful building', 'one of the strangest animals that you can imagine', or something like that. 
If there are more than 1 item, you can use terms showing their relationships with the first item. Such a list could look like so:
['an old castle', 'its tallest tower', 'its roof']. You can also use plural or collective items like 'rats', 'fruit and vegetables', 
or 'a bunch of keys'. You can create a descriptive name like this or just return a simple one-word name for each item. Do not add any explanation 
or comments, just deliver the list of 1-3 items.
"""

instructions_mood = """
You are a story-telling agent responsible for picking the mood for the story. This is your only task, you just provide
the mood and nothing more. The mood can be a simple adjective, like for example 'hilarious', 'sad' or 'emotional', but it can 
be also a noun like for example 'comedy', 'fun', 'terror', or something like that. Another option is to deliver a description 
of the mood like for example 'cheerful at the beginning, but then turning sad' or 'neither cheerful nor sad'.
You can create a descriptive term like this or just return a single word. Do not add any explanation or comments, just deliver the word or 
expression determining the mood of the story.
"""

In [5]:
place_agent = Agent(
    name="Storytelling Place Agent",
    instructions=instructions_place,
    model="gpt-4o-mini"
)

time_agent = Agent(
    name="Storytelling Time Agent",
    instructions=instructions_time,
    model="gpt-4o-mini"
)

characters_agent = Agent(
    name="Storytelling Characters Agent",
    instructions=instructions_characters,
    model="gpt-4o-mini"
)

items_agent = Agent(
    name="Storytelling Items Agent",
    instructions=instructions_items,
    model="gpt-4o-mini"
)

mood_agent = Agent(
    name="Storytelling Mood Agent",
    instructions=instructions_mood,
    model="gpt-4o-mini"
)

In [6]:
message = "Create the element of the story that you're responsible for."

In [7]:
with trace("Parallel story elements"):
    results = await asyncio.gather(
        Runner.run(place_agent, message),
        Runner.run(time_agent, message),
        Runner.run(characters_agent, message),
        Runner.run(items_agent, message),
        Runner.run(mood_agent, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

The Whispering Woods


June of 1920


['Eva', 'her grandfather', 'the village elder']


['a mysterious lantern', 'a swirling fog', 'a secluded path']


Melancholic




## Tools

In [8]:
RESEND_API_KEY = os.environ.get("RESEND_API_KEY")

In [ ]:
# check if the email is sent correctly
def send_test_email(story: str):
    """ Send out an email with the story using Resend """
    
    # Set up email sender, recipient, and content
    from_email = "onboarding@resend.dev"  
    to_email = "prospero.apps@gmail.com"  
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"Prospero <{from_email}>",
        "to": [to_email],
        "subject": "Story of the Day",
        "html": f"<p>{story}</p>"  
    }
    
    # Send email using Resend API
    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
    
    # Check if the request was successful
    print(response.text)
    if 200 <= response.status_code < 300:
        return {"status": "success"}
    else:
        return {"status": "failure", "message": response.text}

send_test_email("wow, test story")

{"id":"0bb6a987-17e2-4b90-8e73-f8a47756f6a4"}


{'status': 'success'}

In [27]:
@function_tool
def send_email(story: str):
    """ Send out an email with the story using Resend """
    
    # Set up email sender, recipient, and content
    from_email = "onboarding@resend.dev"  
    to_email = "prospero.apps@gmail.com"  
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"Prospero <{from_email}>",
        "to": [to_email],
        "subject": "Story of the Day",
        "html": f"<p>{story}</p>"  
    }
    
    try:
        response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
        
        if 200 <= response.status_code < 300:
            return f"SUCCESS: Email with content '{story}' has been sent successfully to {to_email}. Task completed."
        else:
            return f"FAILURE: Failed to send email. Error: {response.text}. Do not retry."
    except Exception as e:
        return f"ERROR: Exception occurred while sending email: {str(e)}. Do not retry."

In [28]:
place_tool = place_agent.as_tool(tool_name="place_tool", tool_description=message)
time_tool = time_agent.as_tool(tool_name="time_tool", tool_description=message)
characters_tool = characters_agent.as_tool(tool_name="characters_tool", tool_description=message)
items_tool = items_agent.as_tool(tool_name="items_tool", tool_description=message)
mood_tool = mood_agent.as_tool(tool_name="mood_tool", tool_description=message)

tools = [place_tool, time_tool, characters_tool, items_tool, mood_tool, send_email]

In [29]:
tools

[FunctionTool(name='place_tool', description="Create the element of the story that you're responsible for.", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'place_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000182A56D1A80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='time_tool', description="Create the element of the story that you're responsible for.", params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'time_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000182A06D4E00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='characters_tool', description="Create the element of the story that you'

## Story Manager

In [48]:
instructions_story = """
You are a story teller. Your task is to create a story using the tools in the tools list. Follow these steps carefully:
1. Use the place_tool tool to generate the place the story is set in.
2. Use the time_tool tool to generate the time the story is set in.
3. Use the characters_tool tool to generate the characters that are featured in the story.
4. Use the items_tool tool to generate the items that are are used in the story.
5. Use the mood_tool tool to generate the mood of the story.

After you have all of these, follow these steps to write the story:
1. Before the introduction part of the story, write what the tools above have generated in the following format, now, VERY, VERY IMPORTANT: 
write each of the following elements on a separate line:
place: here comes the place generated by place_tool
time: here comes the time generated by time_tool
characters: here come the characters generated by characters_tool, separated with commas
items: here come the items generated by items_tool, separated with commas
mood: here comes the mood generated by mood_tool
2. Write the introduction where you say where and when the story is set and what characters are featured in the story. 
If there are more than one character, you don't have to introduce them all in this part, some of them may appear later
in the story. The introduction should be about 50 words long.
3. Write the main part of the story. Think of something interesting that would make a good story. Make sure the mood
of the story that was generated before is maintained throughout the story. Also, make sure all the characters and all 
the items you generated before are used in the story in a sensical way. The story should be consistent. Also, keep in mind
the place and time the story is set in. This may influence the chain of events in the story. This part should be between 200
and 300 words long.
4. Write and ending of the story, but try to make it as unexpected as you can, still keeping it sensical and consistent.
Remember to preserve the mood all the time. The ending should be about 50 words long.
5. Send the story to the recipient using the send_email tool.

IMPORTANT INSTRUCTIONS:
1. Use the send_email tool exactly once with the provided content
2. After calling the tool, check the response
3. If the response indicates SUCCESS, say "Task completed successfully" and STOP
4. If the response indicates FAILURE or ERROR, report the error and STOP
5. Do NOT retry failed email sends
6. Do NOT call the tool multiple times
7. Do NOT ask for confirmation before sending

Your task is complete once you've called the send_email tool and reported the result.
"""

In [49]:
story_manager = Agent(
    name="Story Manager",
    instructions=instructions_story,
    model="gpt-4o-mini",
    tools=tools
)

email_message = "Send an email containing the story to the recipient."

with trace("Story Manager"):
    result = await Runner.run(story_manager, email_message)

print(result)

RunResult:
- Last agent: Agent(name="Story Manager", ...)
- Final output (str):
    Task completed successfully.
- 13 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


## Handoffs

In [50]:
instructions_title = """Your task is to create a title for the story. This title should be a short phrase that captures the essence 
of the story. It's also going to be used as part of the email subject, so it should be catchy and interesting.
"""

instructions_html = """You can convert the text of the story that you are sending via email to HTML. You are given the text of the story 
which might have some markdown and you need to convert it to an HTML version with simple, clear, compelling layout and design.
"""

title_writer = Agent(
    name="Story title writer", 
    instructions=instructions_title, 
    model="gpt-4o-mini"
)

title_tool = title_writer.as_tool(
    tool_name="title_writer", 
    tool_description="Write a title for the story"
)

html_converter = Agent(
    name="HTML story text converter", 
    instructions=instructions_html, 
    model="gpt-4o-mini"
)

html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert the text email body, which is the story, to an HTML email body"
)

In [62]:
@function_tool
def send_html_email(title: str, html_story: str) -> Dict[str, str]:
    """ Send out an email with a specific subject and the story in HTML format using Resend """
    
    # Set up email sender, recipient, and content
    from_email = "onboarding@resend.dev"  
    to_email = "prospero.apps@gmail.com"  
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"Prospero <{from_email}>",
        "to": [to_email],
        "subject": f"Story of the Day - {title}",
        "html": html_story
    }
    
    try:
        response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
        
        if 200 <= response.status_code < 300:
            return f"SUCCESS: Email with content '{html_story}' has been sent successfully to {to_email}. Task completed."
        else:
            return f"FAILURE: Failed to send email. Error: {response.text}. Do not retry."
    except Exception as e:
        return f"ERROR: Exception occurred while sending email: {str(e)}. Do not retry."

In [63]:
email_tools = [title_tool, html_tool, send_html_email]
email_tools

[FunctionTool(name='title_writer', description='Write a title for the story', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'title_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000182A599F100>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert the text email body, which is the story, to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000182A599E480>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with a specific subject and the 

In [64]:
instructions_html_story = """
You are an email formatter and sender. You receive the body of an email to be sent. 
You first use the title_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. 
Finally, you use the send_html_email tool to send the email with the subject and HTML body.
"""

In [65]:
email_agent = Agent(
    name="Email Manager",
    instructions=instructions_html_story,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [79]:
instructions_email = """
You are a story teller. Your task is to create a story using the tools given to you. Follow these steps carefully:
1. Use the place_tool tool to generate the place the story is set in.
2. Use the time_tool tool to generate the time the story is set in.
3. Use the characters_tool tool to generate the characters that are featured in the story.
4. Use the items_tool tool to generate the items that are are used in the story.
5. Use the mood_tool tool to generate the mood of the story.

You can use the tools multiple times if you're not satisfied with the results from the first try.

After you have all of these, follow these steps to write the story:
1. Write what the tools above have generated in the following format, now, VERY, VERY IMPORTANT: 
write each of the following elements on a separate line and MAKE SURE TO USE THE EXACT SAME FORMAT AS SHOWN HERE:
place: here comes the place generated by place_tool,
time: here comes the time generated by time_tool,
characters: here come the characters generated by characters_tool, separated with commas
items: here come the items generated by items_tool, separated with commas
mood: here comes the mood generated by mood_tool.
Then add two blank lines.
2. Write the introduction where you say where and when the story is set and what characters are featured in the story. 
If there are more than one character, you don't have to introduce them all in this part, some of them may appear later
in the story. The introduction should be about 50 words long.
3. Write the main part of the story. Think of something interesting that would make a good story. Make sure the mood
of the story that was generated before is maintained throughout the story. Also, make sure all the characters and all 
the items you generated before are used in the story in a sensical way. The story should be consistent. Also, keep in mind
the place and time the story is set in. This may influence the chain of events in the story. This part should be between 200
and 300 words long.
4. Write and ending of the story, but try to make it as unexpected as you can, still keeping it sensical and consistent.
Remember to preserve the mood all the time. The ending should be about 50 words long.
5. Handoff to the email_agent to format the story to HTML and send it.

IMPORTANT INSTRUCTIONS:
1. Use the send_email tool exactly once with the provided content
2. After calling the tool, check the response
3. If the response indicates SUCCESS, say "Task completed successfully" and STOP
4. If the response indicates FAILURE or ERROR, report the error and STOP
5. Do NOT retry failed email sends
6. Do NOT call the tool multiple times
7. Do NOT ask for confirmation before sending
"""

In [83]:
tell_manager = Agent(
    name="Tell Manager",
    instructions=instructions_email,
    model="gpt-4o-mini",
    tools=tools,
    handoffs=[email_agent]
)

email_message = "Send an email with the subject and HTML body containing the story to the recipient."

with trace("Email Sender"):
    result = await Runner.run(tell_manager, email_message)

print(result)

RunResult:
- Last agent: Agent(name="Tell Manager", ...)
- Final output (str):
    Task completed successfully.
- 14 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
